<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jan 16 15:38:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# nlp aug
!pip install nlpaug transformers

# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased

# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base

# bert uncased large
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1?tf-hub-format=compressed -O blu
!mkdir bert_large_uncased
!tar -xzf blu -C bert_large_uncased



In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import sys
sys.path.extend(['/content/drive/My Drive/GoogleQA'])
import numpy as np
import pandas as pd
import os, gc, codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
import bert_tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
# import optimizer_lamb
tf.get_logger().setLevel('ERROR') 
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
from functools import partial
from scipy.stats import rankdata
from tensorflow.python.framework import ops
# import nlpaug.augmenter.word as naw
# import transformers
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


##Load dataset

In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
# aux_path = 'drive/My Drive/GoogleQA/Web Scrap/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
target_col = train.columns.tolist()[11:42]
q_col, a_col = target_col[:21], target_col[21:]


# Label encoding category and host
class MyLabelEncoder(object):
    """safely handle unknown label"""
    def __init__(self):
        self.mapper = {}
    def fit(self, X):
        uniq_X = np.unique(X)
        # reserve 0 for unknown
        self.mapper = dict(zip(uniq_X, range(1, len(uniq_X) + 1)))
        return self
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    def _map(self, x):
        return self.mapper.get(x, 0)
    def transform(self, X):
        return list(map(self._map, X))

category_encoder = MyLabelEncoder().fit(train['category'])
train['category'] = category_encoder.transform(train['category'])
test['category'] = category_encoder.transform(test['category'])
host_encoder = MyLabelEncoder().fit(train['host'])
train['host'] = host_encoder.transform(train['host'])
test['host'] = host_encoder.transform(test['host'])

# average question targets within the same questions
def q_avg(data_df):
    q_col_med = data_df.groupby(['question_title'], as_index=False)[q_col].mean()
    return data_df.drop(target_col, 1).merge(q_col_med, how='left', on='question_title').drop_duplicates('question_title').reset_index(drop=True)

# assign class weights
def label_wt(data_df, cols, wt1=2, wt2=1.5, wt3=1):
    for col in cols:
        wt = data_df[col].value_counts(normalize=True).rename(col+'_wt').to_frame()
        wt[col+'_wt'] = np.where(wt[col+'_wt']<=0.01, wt1, np.where(wt[col+'_wt']<=0.05, wt2, wt3))
        data_df = data_df.merge(wt, 'left', left_on=col, right_on=wt.index)
    return data_df

# train, train_q, train_a = label_wt(train, target_col), label_wt(train_q, q_col), label_wt(train_a, a_col)
col_wt, q_col_wt, a_col_wt = [x+'_wt' for x in target_col], [x+'_wt' for x in q_col], [x+'_wt' for x in a_col]

## Parameter setting

In [0]:
MAXLEN = 512 
BATCH_SIZE = 4
NUM_EPOCHS = 3
NUM_CLASSES = 30
NUM_AUX = 6
LR = 5e-5
MIN_LR = 0
model_path = 'drive/My Drive/GoogleQA/bert_base_uncased' 
save_path = 'bbu10'
save_model = 'bbu10' 
CASED = False


## Bert tokenizer

In [0]:
def _trim_input(title, question, answer, max_sequence_length=MAXLEN,
                t_max_len=15, q_max_len=245, a_max_len=248):
    t_len = len(title)
    q_len = len(question)
    a_len = len(answer)
    if (t_len + q_len + a_len +6) > max_sequence_length:
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + np.floor((t_max_len - t_len) / 2)
            q_max_len = q_max_len + np.ceil((t_max_len - t_len) / 2)
        else:
            t_new_len = t_max_len
        if a_max_len > a_len:
            a_new_len = a_len
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
        if t_new_len + a_new_len + q_new_len +6 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"
                    % (max_sequence_length, (t_new_len + a_new_len + q_new_len +6)))
        head_t_new_len = int(0.25 * t_new_len)
        tail_t_new_len = int(t_new_len - head_t_new_len)

        head_q_new_len = int(0.25 * q_new_len)
        tail_q_new_len = int(q_new_len - head_q_new_len)

        head_a_new_len = int(0.25 * a_new_len)
        tail_a_new_len = int(a_new_len - head_a_new_len)

        title = title[:head_t_new_len] + title[-tail_t_new_len:]
        question = question[:head_q_new_len] + question[-tail_q_new_len:]
        answer = answer[:head_a_new_len] + answer[-tail_a_new_len:]

    return title, question, answer

tokenizer = bert_tokenizer.FullTokenizer(os.path.join(model_path, 'assets/vocab.txt'))
for token in ['[QT]', '[QB]', '[AN]']:
    tokenizer.vocab[token] = len(tokenizer.vocab)
# for category in np.unique(train['category']):
#     tokenizer.vocab[f'[{category}]'] = len(tokenizer.vocab)
# for host in np.unique(train['host']):
#     tokenizer.vocab[f'[{host}]'] = len(tokenizer.vocab)
# tokenizer.vocab['[category_unk]'] = len(tokenizer.vocab)
# tokenizer.vocab['[host_unk]'] = len(tokenizer.vocab)

def q_trim(tokens, max_length=MAXLEN):
# def q_trim(q_, a_, max_length=MAXLEN):
    if len(tokens) > max_length:
        head_length = int(0.25 * max_length)
        tail_length = max_length - head_length
        tokens = tokens[:head_length] + tokens[-tail_length:]
    return tokens
    # if len(q_) >= int(max_length*0.7):
    #     head_length = int(0.25 * int(max_length*0.7))
    #     tail_length = int(max_length*0.7) - head_length
    #     q_ = q_[:head_length] + q_[-tail_length:]

    # if len(q_) + len(a_) > max_length:
    #     a_length = max_length - len(q_)
    #     head_a_length = int(0.25*a_length)
    #     tail_a_length = a_length - head_a_length
    #     a_ = a_[:head_a_length] + a_[-tail_a_length:]
    # return q_, a_

def a_trim(q_, a_, max_length=MAXLEN):
    a_max_length = int(max_length*0.75)
    if len(a_) >= a_max_length:
        head_length = int(0.25 * a_max_length)
        tail_length = a_max_length - head_length
        a_ = a_[:head_length] + a_[-tail_length:]

    if len(q_) + len(a_) > max_length:
        q_length = max_length - len(a_)
        head_q_length = int(0.25*q_length)
        tail_q_length = q_length - head_q_length
        q_ = q_[:head_q_length] + q_[-tail_q_length:]
    return q_, a_

## Convert data to inputs

In [0]:
def convert_data(data_df, model, targets, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    for i in tqdm(range(len(data_df))):
        a = tokenizer.tokenize(data_df.loc[i, 'question_title'])
        b = tokenizer.tokenize(data_df.loc[i, 'question_body'])
        c = tokenizer.tokenize(data_df.loc[i, 'answer'])
        if model == 'qa':
            a, b, c = _trim_input(a, b, c, t_max_len=15, q_max_len=244, a_max_len=247)
            question = tokenizer.convert_tokens_to_ids(['[CLS]']+['[QT]']+a+['[QB]']+b+['[SEP]'])
            answer = tokenizer.convert_tokens_to_ids(['[AN]']+c+['[SEP]'])
            ids.append(question+answer)
            segments.append([0] * len(question) + [1] * len(answer))
        if model == 'q':
            question = tokenizer.convert_tokens_to_ids(['[CLS]']+['[QT]']+a+['[SEP]']+['[QB]']+b+['[SEP]'])
            # answer = tokenizer.convert_tokens_to_ids(['[AN]']+c+['[SEP]'])
            question_trim = q_trim(question)
            ids.append(question_trim)
            segments.append([0] * len(question_trim))
        if model == 'a':
            question = tokenizer.convert_tokens_to_ids(['[CLS]']+['[QT]']+a+['[QB]']+b+['[SEP]'])
            answer = tokenizer.convert_tokens_to_ids(['[AN]']+c+['[SEP]'])         
            question_trim, answer_trim = a_trim(question, answer)
            ids.append(question_trim + answer_trim)
            segments.append([0] * len(question_trim) + [1] * len(answer_trim))
    aux = data_df[['category', 'host']]    
    if branch == 'training':
        targets = data_df[targets]
        return [ids, segments], np.array(aux, dtype='int32'), np.array(targets)
        # return [ids, segments], np.array(targets)
    else:
        return [ids, segments], np.array(aux, dtype='int32')
        # return [ids, segments]

## Data Generator

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train', multi_target=True):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        self.multi_target=multi_target
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])

                aux = np.array([x[2][0] for x in d]).reshape(-1, 1)
                aux = tf.convert_to_tensor(aux, dtype=tf.int32)
                aux2 = np.array([x[2][1] for x in d]).reshape(-1, 1)
                aux2 = tf.convert_to_tensor(aux2, dtype=tf.int32)
                if self.branch != 'train':
                    yield [X1, X2, X3, aux, aux2]
                else:                    
                    if self.multi_target:
                        Y = []
                        num_targets = int(d[0][3].shape[0] / 2)
                        for i in range(num_targets):
                            Y.append(tf.convert_to_tensor([x[3][[i, i+num_targets]] for x in d], dtype=tf.float32))
                    else:
                        Y = tf.convert_to_tensor([x[3] for x in d], dtype=tf.float32)
                    yield [X1, X2, X3, aux, aux2], Y


## Build model structure

In [0]:
@tf.custom_gradient
def transfer(x):
    def grad_fn(dy):
        return dy
    return tf.floor(x*1000)/1000, grad_fn

# Define custom py_func which takes also a grad op as argument:
def py_func(func, inp, Tout, name=None, grad=None):
    
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    
    tf.RegisterGradient(rnd_name)(grad)  
    g = tf.compat.v1.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_function(func, inp, Tout, name=name)

# Def custom square function using np.square instead of tf.square:
def myrankdata(x, name=None):
    
    with ops.op_scope([x], name, "MyGrad") as name:
        rank = py_func(rankdata,
                        [x],
                        [tf.float32],
                        name=name,
                        grad=_MyGrad)  
        return rank

# Actual gradient:
def _MyGrad(op, grad):
    return grad * op.inputs[0]  

def custom_spearman(multi_target):
    def rhos(y_true, y_pred):
        a = y_true[:, 0]
        b = y_pred
        rank_a, rank_b = myrankdata(a)[0], myrankdata(b)[0]
        rho = 1 - 6 * tf.reduce_sum((rank_a - rank_b) ** 2) / (BATCH_SIZE ** 3 - BATCH_SIZE)
        rho = K.maximum(rho, -1.0)
        if multi_target:
            return 1 - K.square(rho) #* y_true[:, 1]
        else:
            return 1-rho
    return rhos

def custom_pearson(multi_target):
    def rhos(y_true, y_pred):
        xm, ym = y_true-K.mean(y_true), y_pred-K.mean(y_pred)
        r_num = K.sum(tf.multiply(xm, ym))
        r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
        r = r_num / r_den
        r = K.maximum(K.minimum(r, 1.0), -1.0)
        if multi_target:
            return (1 - K.square(r)) * y_true[:, 1]
        else:
            return 1 - K.square(r)
    return rhos
def custom_loss(y_true, y_pred):
    return keras.metrics.binary_crossentropy(tf.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]

def model_build(len_train, num_targets, dropout=0.2, lr=LR, epochs=NUM_EPOCHS, bert_trainable=True, multi_target=True):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    # global LR
    global MAXLEN
    global model_path
    global NUM_AUX
 
    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")
    input_category = keras.layers.Input((1,), dtype=tf.int32, name='input_category')
    input_host = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_host')

    bert_layer = hub.KerasLayer(model_path, trainable=bert_trainable)
    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    q_inputs = keras.layers.Lambda(lambda x: K.clip(x, 1e-6, None) ** 3)(q_inputs)
    q_outputs1 = keras.layers.GlobalAveragePooling1D()(q_inputs)
    q_outputs1 = keras.layers.Dropout(dropout)(q_outputs1 ** (1./3))

    category_emb = keras.layers.Embedding(input_dim=len(category_encoder.mapper)+1, output_dim=32)(input_category)
    category_emb = keras.layers.SpatialDropout1D(0.1)(category_emb)

    host_emb = keras.layers.Embedding(input_dim=len(host_encoder.mapper)+2, output_dim=32)(input_host)
    host_emb = keras.layers.SpatialDropout1D(0.1)(host_emb)

    features_dense = keras.layers.concatenate([category_emb, host_emb], axis=1)
    features_dense = keras.layers.Flatten()(features_dense)

    dense = keras.layers.concatenate([q_outputs1, features_dense])

    if multi_target:
        # outputs = [keras.layers.Lambda(transfer)(keras.layers.Dense(1, activation='sigmoid')(dense)) for _ in range(num_targets)]
        outputs = [keras.layers.Dense(1, activation='sigmoid')(dense) for _ in range(num_targets)]
        loss_func = [custom_loss]*num_targets

    else:
        # outputs = keras.layers.Lambda(transfer)(keras.layers.Dense(num_targets, activation='sigmoid')(dense))
        outputs = keras.layers.Dense(num_targets, activation='sigmoid')(dense)
        loss_func = ['binary_crossentropy']
    # model = keras.Model([q_in, q2_in, q3_in], outputs)
    model = keras.Model([q_in, q2_in, q3_in, input_category, input_host], outputs)


    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=epochs,
    )

    model.compile(
        loss=loss_func,
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=lr,
            min_lr=MIN_LR,
            ),
        # optimizer = optimizer_lamb.LAMB(learning_rate=lr)
        )


    return model

## train model

In [0]:
# kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1627)
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    # rho_df = pd.DataFrame(dict(col=cols, rhos=rhos))
    return np.nanmean(rhos)#, rho_df

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, save_model, rho_init=-1, multi_target=True):
        global NUM_EPOCHS
        super().__init__()
        self.rho = []
        self.model_weights = []
        self.save_model = save_model
        self.multi_target = multi_target
        self.rho_init = rho_init
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        if self.multi_target:
            val_pred = np.array(val_pred).squeeze().T
        nunique = [i for i in range(val_pred.shape[1]) if len(np.unique(val_pred[:, i]))==1]
        if len(nunique)>0:
            print(f'{len(nunique)} cols with monotonous preds!')
        score = compute_spearmanr(val_pred, val_y)
        print('Spearman - {:.5f}'.format(score))
        self.rho.append(score)
        self.model_weights.append(self.model.get_weights())
        # if score > self.rho:
        #     self.rho = score
        #     if self.stage==2:
        #         print('--Save Model--')
        #         self.model.save_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-{:}.h5'.format(save_path, self.save_model, i, self.stage))
        #     elif self.stage==1:
        #         print('--Tmp Save--')
        #         self.model.save_weights('model.h5')
    def on_train_end(self, logs=None):
        """
        Weighted average of weights from two best epochs

        """
        rho_selected = np.argsort(self.rho)[-2:]
        print(f'Best epoch {rho_selected[1]+1} and second best epoch {rho_selected[0]+1} selected')
        swa_weights = [x * 0.3 + y * 0.7 for x, y in zip(self.model_weights[rho_selected[0]], self.model_weights[rho_selected[1]])]
        self.model.set_weights(swa_weights)
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        if self.multi_target:
            val_pred = np.array(val_pred).squeeze().T
        nunique = [i for i in range(val_pred.shape[1]) if len(np.unique(val_pred[:, i]))==1]
        if len(nunique)>0:
            print(f'{len(nunique)} cols with monotonous preds!')
        score = compute_spearmanr(val_pred, val_y)
        print('SWD Model Spearman - {:.5f}'.format(score))
        if score>self.rho_init:
            self.rho_init = score
            print('Save SWD Weights')
            self.model.save_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, self.save_model, i))
        del self.rho, self.model_weights, swa_weights
        gc.collect()




### QA combined Model

In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_title)]
multi_target = True

for i, (tr_idx, val_idx) in enumerate(idx[7:], 8):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr = label_wt(tr, target_col)
    tr_x, tr_aux, tr_y = convert_data(tr, model='qa', targets=target_col+col_wt)
    val_x, val_aux, val_y = convert_data(val, model='qa', targets=target_col)

    if sum(val[q_col].nunique()==1)>0:
        print(f'\n{sum(val[q_col].nunique()==1)} with Monotonous Label Detected!\n')

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_aux, tr_y)), multi_target=multi_target)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux)), branch='valid', multi_target=multi_target)
    
    # stage 1 fine tunning
    ieval = IntervalEval(save_model=save_model, multi_target=multi_target)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(target_col), 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    rho_stage1 = ieval.rho_init
    gc.collect()

    # stage 2 fine tunning
    ieval = IntervalEval(save_model=save_model, multi_target=multi_target, rho_init=rho_stage1)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(target_col), 
        lr=1e-4, 
        epochs=4, 
        bert_trainable=False, 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, save_model, i))

    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

### Q Model

In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_q = [x for x in kf.split(train, groups=train.question_title)]
multi_target = False

for i, (tr_idx, val_idx) in enumerate(idx_q[:1], 1):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr = q_avg(tr)
    # tr = label_wt(tr, q_col)
    # tr = data_aug(tr, model='q')
    tr_x, tr_aux, tr_y = convert_data(tr, model='q', targets=q_col)
    val_x, val_aux, val_y = convert_data(val, model='q', targets=q_col)

    if sum(val[q_col].nunique()==1)>0:
        print(f'\n{sum(val[q_col].nunique()==1)} with Monotonous Label Detected!\n')

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_aux, tr_y)), multi_target=multi_target)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux)), branch='valid', multi_target=multi_target)
    
    # stage 1 fine tunning
    ieval = IntervalEval(save_model=save_model+'_q', multi_target=multi_target)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(q_col), 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )   
    rho_stage1 = ieval.rho_init

    # stage 2 fine tunning
    ieval = IntervalEval(save_model=save_model+'_q', multi_target=multi_target, rho_init=rho_stage1)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(q_col), 
        lr=1e-4, 
        epochs=4, 
        bert_trainable=False, 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, save_model+'_q', i))
    # model.load_weights('model.h5')

    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

  1%|          | 22/3175 [00:00<00:14, 218.19it/s]


Fold - 1



100%|██████████| 1216/1216 [00:05<00:00, 232.54it/s]


Train for 794 steps
Epoch 1/3
304/304 [==============================] - 23s 75ms/step
Spearman - 0.40149
794/794 [==============================] - 248s 312ms/step - loss: 0.4119
Epoch 2/3
304/304 [==============================] - 21s 71ms/step
Spearman - 0.42111
794/794 [==============================] - 215s 271ms/step - loss: 0.3619
Epoch 3/3
162/794 [=====>........................] - ETA: 2:35 - loss: 0.3387

### A Model

In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_a = [x for x in kf.split(train_a, groups=train_a.question_title)]
multi_target = True

for i, (tr_idx, val_idx) in enumerate(idx_a[:1], 1):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train_a.loc[tr_idx], train_a.loc[val_idx]
    tr = label_wt(tr, a_col)
    # tr = data_aug(tr, model='a')
    tr_x, tr_aux, tr_y = convert_data(tr, model='a', targets=a_col+a_col_wt)
    val_x, val_aux, val_y = convert_data(val, model='a', targets=a_col)

    if sum(val[a_col].nunique()==1)>0:
        print(f'\n{sum(val[a_col].nunique()==1)} with Monotonous Label Detected!\n')

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_aux, tr_y)), multi_target=multi_target)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux)), branch='valid', multi_target=multi_target)
    
    # stage 1 fine tunning
    ieval = IntervalEval(save_model=save_model+'_a', multi_target=multi_target)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(a_col),
        multi_target=multi_target,
        dropout=0.1,
        )
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval],
    )   
    rho_stage1 = ieval.rho_init
    gc.collect()

    # stage 2 fine tunning
    ieval = IntervalEval(save_model=save_model+'_a', multi_target=multi_target, rho_init=rho_stage1)
    model = model_build(
        len_train=len(tr), 
        num_targets=len(a_col), 
        lr=1e-4, 
        epochs=4, 
        bert_trainable=False, 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, save_model+'_a', i))
    # model.load_weights('model.h5')

    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval],
    )

    gc.collect()

Train for 1216 steps
Epoch 1/3
304/304 [==============================] - 30s 100ms/step
9 cols with monotonous preds!
Spearman - nan
1216/1216 [==============================] - 466s 383ms/step - loss: 5.0742 - lambda_loss: 0.5737 - lambda_1_loss: 0.5337 - lambda_2_loss: 0.4440 - lambda_3_loss: 0.3874 - lambda_4_loss: 0.7006 - lambda_5_loss: 0.6472 - lambda_6_loss: 0.4977 - lambda_7_loss: 0.6625 - lambda_8_loss: 0.6274
Epoch 2/3
 29/304 [=>............................] - ETA: 26s

KeyboardInterrupt: ignored

##oof prediction

In [0]:
# oof-prediction
multi_target = True
model_qa = model_build(len_train=len(train), num_targets=len(target_col), bert_trainable=False, multi_target=multi_target)
model_q = model_build(len_train=len(train), num_targets=len(q_col), bert_trainable=False, multi_target=multi_target)
model_a = model_build(len_train=len(train), num_targets=len(a_col), bert_trainable=False, multi_target=multi_target)
oof_pred_qa = train[['qa_id']+target_col].copy()
oof_pred_qa[target_col] = 0
oof_pred_q = train_q[['qa_id']+q_col].copy()
oof_pred_q[q_col] = 0
oof_pred_a = train_a[['qa_id']+a_col].copy()
oof_pred_a[a_col] = 0

kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx_qa = [x for x in kf.split(train, groups=train.question_title)]
for i, (tr_idx, val_idx) in enumerate(idx_qa, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    val_x, val_aux, val_y = convert_data(val, 'qa', targets=target_col+col_wt)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux, val_y)), branch='valid', multi_target=multi_target)
    model_qa.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}.h5'.format(save_path, save_model, i))
    val_pred_qa = model_qa.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred_qa.loc[val_idx, target_col] += np.array(val_pred_qa).squeeze().T / 2

    score = compute_spearmanr(val_y, np.array(val_pred_qa).squeeze().T)
    print('\nQA - Spearman - {:.5f}\n'.format(score))


kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_q = [x for x in kf.split(train, groups=train.question_title)]
for i, (tr_idx, val_idx) in enumerate(idx_q, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    # val = add_host_category(val)
    val_x, val_aux, val_y = convert_data(val, 'q', targets=q_col+q_col_wt)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux, val_y)), branch='valid', multi_target=multi_target)
    model_q.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}.h5'.format(save_path, save_model+'_q', i))
    val_pred_q = model_q.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred_q.loc[val_idx, q_col] += np.array(val_pred_q).squeeze().T# / 2

    score = compute_spearmanr(val_y, np.array(val_pred_q).squeeze().T)
    print('\nQ - Spearman - {:.5f}\n'.format(score))

idx_a = [x for x in kf.split(train, groups=train.question_title)]
for i, (tr_idx, val_idx) in enumerate(idx_a, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    val_x, val_aux, val_y = convert_data(val, 'a', targets=a_col+a_col_wt)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux, val_y)), branch='valid', multi_target=multi_target)
    model_a.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}.h5'.format(save_path, save_model+'_a', i))
    val_pred_a = model_a.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred_a.loc[val_idx, a_col] += np.array(val_pred_a).squeeze().T
    
    score = compute_spearmanr(val_y, np.array(val_pred_a).squeeze().T)
    print('\nA - Spearman - {:.5f}\n'.format(score))

    gc.collect()

oof_pred = oof_pred_qa[target_col].values * 0.4 + np.concatenate((oof_pred_q[q_col].values, oof_pred_a[a_col].values), axis=1) * 0.6
np.save('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu10.npy', oof_pred)

  3%|▎         | 24/717 [00:00<00:03, 226.23it/s]


Fold - 1



100%|██████████| 717/717 [00:03<00:00, 222.59it/s]


180/180 [==============================] - 15s 82ms/step


  0%|          | 0/717 [00:00<?, ?it/s]

...

179/179 [==============================] - 13s 72ms/step


  0%|          | 0/716 [00:00<?, ?it/s]


Q - Spearman - 0.40732


Fold - 5



100%|██████████| 716/716 [00:03<00:00, 230.35it/s]


179/179 [==============================] - 13s 73ms/step

Q - Spearman - 0.40488



## oof-prediction hyperparameterize

### Just use "90" transform

In [0]:
compute_spearmanr(oof_pred_[a_col].values, train_a[a_col].values)


0.3678945323743401

In [0]:
# bbu3: 0.42130
# bbu4: 0.43406
# bbu5: 0.42747
# bbu6: 0.43264
# bbu9: 0.43900
# bbu4 + bbu9: 0.44547
# lindada*0.4 + bbu9*0.6: 0.44867
oof_pred4 = pd.read_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu4.csv')
oof_pred9 = pd.read_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu9.csv')
oof_pred = oof_pred4[target_col] *0.4 + oof_pred9[target_col] *0.6
score_qa = compute_spearmanr(np.floor(oof_pred_qa[target_col].values*90)/90, train[target_col].values)
score_q = compute_spearmanr(np.floor(oof_pred_q[q_col].values*90)/90, train_q[q_col].values)
score_a = compute_spearmanr(np.floor(oof_pred_a[a_col].values*90)/90, train_a[a_col].values)
# print('Total Score: {:.5f}'.format(score_q*0.7+score_a*0.3))

# just using "90" transformation
scores = []
for col, col_trues, col_pred in zip(target_col, train[target_col].values.T, (np.floor(oof_pred_qa[target_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))
    scores.append(corr)
hyper1_qa_combined = pd.DataFrame(dict(col = target_col, score1 = scores))


scores = []
for col, col_trues, col_pred in zip(q_col, train[q_col].values.T, (np.floor(oof_pred_q[q_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))
    scores.append(corr)
hyper1_q = pd.DataFrame(dict(col=q_col, score1=scores))

scores = []
for col, col_trues, col_pred in zip(a_col, train[a_col].values.T, (np.floor(oof_pred_a[a_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))
    scores.append(corr)
print(np.mean(scores))
hyper1_a = pd.DataFrame(dict(col=a_col, score1=scores))
hyper1_qa = pd.concat([hyper1_q, hyper1_a], sort=False, ignore_index=True)

question_asker_intent_understanding - 0.38051
question_body_critical - 0.64965
question_conversational - 0.45531
question_expect_short_answer - 0.31746
question_fact_seeking - 0.38686
question_has_commonly_accepted_answer - 0.44411
question_interestingness_others - 0.35485
question_interestingness_self - 0.49944
question_multi_intent - 0.60302
question_not_really_a_question - 0.13203
question_opinion_seeking - 0.49888
question_type_choice - 0.75529
question_type_compare - 0.40765
question_type_consequence - 0.22527
question_type_definition - 0.48513
question_type_entity - 0.50119
question_type_instructions - 0.79027
question_type_procedure - 0.36777
question_type_reason_explanation - 0.68575
question_type_spelling - 0.24185
question_well_written - 0.53197
answer_helpful - 0.23650
answer_level_of_information - 0.43432
answer_plausible - 0.16032
answer_relevance - 0.19304
answer_satisfaction - 0.33071
answer_type_instructions - 0.76870
answer_type_procedure - 0.29403
answer_type_reason_e

### Use lower (to 0) or upper (to 1) resetting

In [0]:
# bbu3: 0.44169
# bbu4: 0.45424
# bbu5: 0.45451
# bbu6: 0.45252
# bbu7: 0.45296
# bbu9: 0.45732
# bbu4 + bbu9: 0.46861 (LB: 0.457)
# bbu4*0.4 + bbu9 * 0.6: 0.47113
# lindada*0.4 + bbu9*0.6: 0.46791
# bbu10 qa combined: 0.46153 (LB: 0.448)

def f_min(params, pred, train, col):
    tmp = pred.copy()[col].values
    if params['choice'] == 'low':
        loweridx = tmp.argsort()[:params['pct']]
        tmp[loweridx] = 0
    else:
        upperidx = tmp.argsort()[-params['pct']:]
        tmp[upperidx] = 1
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

score, choice, pct = [], [], []
for col in target_col:
    corrparams = {
    'pct': hp.randint('pct', len(oof_pred_qa)),
    'choice': hp.choice('choice', ['low', 'up'])
    }
    f = partial(f_min, pred=oof_pred_qa, train=train, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - {} - pct: {:.3f}'.format(
        col, 
        -trials.best_trial['result']['loss'],
        ['low', 'up'][best['choice']],
        best['pct']/len(oof_pred_qa),
        # best['upper']/len(oof_pred)
        ))
    score.append(-trials.best_trial['result']['loss'])
    pct.append(round(best['pct']/len(oof_pred_qa), 3))
    choice.append(['low', 'up'][best['choice']])
hyper2_qa_combined = pd.DataFrame(dict(col = target_col, pct = pct, choice=choice, score2 = score))


score, choice, pct = [], [], []
for col in q_col:
    corrparams = {
    'pct': hp.randint('pct', len(oof_pred_q)),
    'choice': hp.choice('choice', ['low', 'up'])
    }
    f = partial(f_min, pred=oof_pred_q, train=train_q, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - {} - pct: {:.3f}'.format(
        col, 
        -trials.best_trial['result']['loss'],
        ['low', 'up'][best['choice']],
        best['pct']/len(oof_pred_q),
        # best['upper']/len(oof_pred)
        ))
    score.append(-trials.best_trial['result']['loss'])
    pct.append(round(best['pct']/len(oof_pred_q), 3))
    choice.append(['low', 'up'][best['choice']])
hyper2_q = pd.DataFrame(dict(col = q_col, pct = pct, choice=choice, score2 = score))
hyper2_q.score2.mean()

score, choice, pct = [], [], []
for col in a_col:
    corrparams = {
    'pct': hp.randint('pct', len(oof_pred_a)),
    'choice': hp.choice('choice', ['low', 'up'])
    }
    f = partial(f_min, pred=oof_pred_a, train=train_a, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - {} - pct: {:.3f}'.format(
        col, 
        -trials.best_trial['result']['loss'],
        ['low', 'up'][best['choice']],
        best['pct']/len(oof_pred_a),
        # best['upper']/len(oof_pred)
        ))
    score.append(-trials.best_trial['result']['loss'])
    pct.append(round(best['pct']/len(oof_pred_a), 3))
    choice.append(['low', 'up'][best['choice']])

print(np.mean(score))

hyper2_a = pd.DataFrame(dict(col = a_col, pct = pct, choice=choice, score2 = score))
hyper2_qa = pd.concat([hyper2_q, hyper2_a], sort=False, ignore_index=True)

question_asker_intent_understanding - 0.38184 - low - pct: 0.005
question_body_critical - 0.65004 - low - pct: 0.005
question_conversational - 0.51796 - low - pct: 0.881
question_expect_short_answer - 0.31758 - low - pct: 0.005
question_fact_seeking - 0.3871 - low - pct: 0.005
question_has_commonly_accepted_answer - 0.49328 - up - pct: 0.794
question_interestingness_others - 0.36194 - low - pct: 0.495
question_interestingness_self - 0.51426 - low - pct: 0.620
question_multi_intent - 0.61359 - low - pct: 0.555
question_not_really_a_question - 0.14429 - low - pct: 0.928
question_opinion_seeking - 0.49871 - low - pct: 0.005
question_type_choice - 0.77419 - low - pct: 0.484
question_type_compare - 0.53411 - low - pct: 0.928
question_type_consequence - 0.26367 - low - pct: 0.953
question_type_definition - 0.63317 - low - pct: 0.928
question_type_entity - 0.61702 - low - pct: 0.881
question_type_instructions - 0.79948 - up - pct: 0.354
question_type_procedure - 0.36789 - low - pct: 0.005
que

### Use n-binning

In [0]:
# bbu3: 0.43042
# bbu4: 0.44542

def f1_min(params, col):
    tmp = oof_pred.copy()[col].values
    paramlist = [params['p'+str(i)] for i in range(len(params))]
    bins = np.percentile(tmp, paramlist)
    tmp = np.digitize(tmp, np.sort(bins)) / len(bins)
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

def f2_min(params, col):
    cutoff = {'p'+str(i): hp.randint('p'+str(i), 101) for i in range(params['num_bin'])}
    f = partial(f1_min, col=col)
    trials = Trials()
    best = fmin(f, cutoff, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, 
                show_progressbar=False)
    # print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score = -trials.best_trial['result']['loss']
    return {'loss': -round(score, 5), 'status': STATUS_OK, 'cutoff': list(best.values())}

score, cutoffs = [], []
for col in target_col:
    numbin = {'num_bin': hp.randint('num_bin', 19)+2}
    f = partial(f2_min, col=col)
    trials = Trials()
    best = fmin(f, numbin, tpe.suggest, 10, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score.append(-trials.best_trial['result']['loss'])
    cutoffs.append(trials.best_trial['result']['cutoff'])

print(np.mean(score))
hyper3 = pd.DataFrame(dict(col = target_col, cutoffs_pct = cutoffs, score3 = score))


### Combine "90" transform and 0/1 resetting

In [0]:
# bbu 3: 0.44462
# bbu 4: 0.45486 (LB: 0.444)
# bbu 5: 0.44808 (LB: 0.441)
# bbu 6: 0.4539
# bbu 7: 0.45297
# bbu 9: 0.45735 (LB: 0.451)
# bbu 9 *0.6 + lindada *0.4: 0.46795
# bbu 10_qa_combined: 0.46153
hyper = hyper1_qa_combined.merge(hyper2_qa_combined, on='col')#.merge(hyper3, on='col')
hyper.to_csv('drive/My Drive/GoogleQA/Models/bbu10/hyper_bbu10_qa_combined.csv', index=False)
hyper['score'] = hyper[['score1', 'score2']].max(axis=1)
hyper['score'].mean()

0.4615320591221413

### use hyper_bbu4

In [0]:
hyper = pd.read_csv('drive/My Drive/GoogleQA/Data/hyper_bbu4.csv')
col1 = hyper.loc[hyper[['score1', 'score2']].max(axis=1) == hyper.score1, 'col'].tolist()
col2 = hyper.loc[hyper[['score1', 'score2']].max(axis=1) == hyper.score2, 'col'].tolist()
pred = np.copy(oof_pred)

for col in col1:
    colidx = target_col.index(col)
    pred[:, colidx] = (pred[:, colidx]//(1/90))/90

for col in col2:
    if hyper.loc[hyper.col==col, 'pct'].values == 1:
        print(f'{col} - minus 0.005')
        pct = hyper.loc[hyper.col==col, 'pct'] - 0.005
    elif hyper.loc[hyper.col==col, 'pct'].values == 0:
        print(f'{col} - plus 0.005')
        pct = hyper.loc[hyper.col==col, 'pct'] + 0.005
    else:
        print(f'{col} - use original')
        pct = hyper.loc[hyper.col==col, 'pct']
        
    changerow = int(len(test) * pct)
    colidx = target_col.index(col)
    
    if hyper.loc[hyper.col==col, 'choice'].values =='low':
        rowidx = pred[:, colidx].argsort()[:changerow]
        pred[rowidx, colidx] = 0
    elif hyper.loc[hyper.col==col, 'choice'].values =='up':
        rowidx = pred[:, colidx].argsort()[-changerow:]
        pred[rowidx, colidx] = 1
    else:
        print('Wrong!')

question_asker_intent_understanding - use original
question_conversational - use original
question_has_commonly_accepted_answer - use original
question_interestingness_others - use original
question_interestingness_self - use original
question_multi_intent - use original
question_not_really_a_question - use original
question_opinion_seeking - plus 0.005
question_type_choice - use original
question_type_compare - use original
question_type_consequence - use original
question_type_definition - use original
question_type_entity - use original
question_type_instructions - use original
question_type_procedure - use original
question_type_reason_explanation - use original
question_type_spelling - minus 0.005
question_well_written - use original
answer_helpful - plus 0.005
answer_plausible - use original
answer_relevance - use original
answer_satisfaction - plus 0.005
answer_type_instructions - use original
answer_type_procedure - plus 0.005
answer_type_reason_explanation - plus 0.005


## test prediction

## Data augmentation

In [0]:
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", top_k=100, aug_p=0.3)
def row_aug(data_df):
    q_id, a_id = [], []
    for col in target_col:
        a = data_df[col].value_counts(normalize=True)
        b = a[a < 0.01].index.tolist()
        c = data_df.loc[data_df[col].isin(b), 'qa_id'].tolist()
        if 'question' in col:
            q_id.extend(c)
        else:
            a_id.extend(c)
    return list(set(q_id)), list(set(a_id))

def data_aug(data_df, model):
    q_id, a_id = row_aug(data_df)
    if model == 'q':
        for row in tqdm(q_id):    
            qt = data_df.loc[data_df.qa_id==row, 'question_title'].to_string(index=False)
            qb = data_df.loc[data_df.qa_id==row, 'question_body'].to_string(index=False)
            # for _ in range(1):
            tmp = data_df[data_df.qa_id==row].copy()
            tmp['question_title'] = aug.augment(qt)
            tmp['question_body'] = aug.augment(qb)
            data_df = pd.concat([data_df, tmp], ignore_index=True)
    else:
        for row in tqdm(a_id):    
            an = data_df.loc[data_df.qa_id==row, 'answer'].to_string(index=False)
            # for _ in range(1):
            tmp = data_df[data_df.qa_id==row].copy()
            tmp['answer'] = aug.augment(an)
            data_df = pd.concat([data_df, tmp], ignore_index=True)
    return data_df

In [0]:
test_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/test_oof_pred.csv')
hyper = pd.read_csv('drive/My Drive/GoogleQA/Data/hyper.csv')
pred = test_pred[target_col].copy().values

In [0]:
col1 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score1, 'col'].tolist()
col2 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score2, 'col'].tolist()
col3 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score3, 'col'].tolist()


for col in col1:
    colidx = target_col.index(col)
    pred[:, colidx] = (pred[:, colidx]//(1/90))/90

for col in col2:
    if hyper.loc[hyper.col==col, 'lower'].values == 1:
        pct = hyper.loc[hyper.col==col, 'lower'] - 0.01
    else:
        pct = hyper.loc[hyper.col==col, 'lower']
    changerow = int(len(test) * pct)
    colidx = target_col.index(col)
    rowidx = pred[:, colidx].argsort()[:changerow]
    pred[rowidx, colidx] = 0
    
for col in col3:
    cutoffs_pct = [int(x) for x in hyper.loc[hyper.col==col, 'cutoffs_pct'].strip('[]').split(',')]
    colidx = target_col.index(col)
    bins = np.percentile(pred[:, colidx], list(set(cutoffs_pct)))
    pred[:, colidx] = np.digitize(pred[:, colidx], np.sort(bins)) / len(bins)


['question_body_critical', 'answer_level_of_information']

## Experimental engineered features

In [0]:
# def aux_features(data_df):
#     data_df["n_question_body"] = data_df["question_body"].apply(lambda x: len(x.strip())) / 10000
#     data_df["n_question_title"] = data_df["question_title"].apply(lambda x: len(x.strip())) / 100
#     data_df["n_answer"] = data_df["answer"].apply(lambda x: len(x.strip())) / 10000
#     data_df["n_word_question_body"] = data_df["question_body"].apply(lambda x: len(x.split())) / 100
#     data_df["n_word_question_title"] = data_df["question_title"].apply(lambda x: len(x.split())) / 100
#     data_df["n_word_answer"] = data_df["answer"].apply(lambda x: len(x.split())) / 100
#     data_df["is_question_text_in_title"] = data_df["question_title"].apply(lambda x: float(
#         x.split()[0] in ["Do", "Can", "What", "How", "Is", "Which", "When", "Where", "can", "what", "how", "is",
#                             "which", "What's", "does", "Does", "Are", "are", "Should", "Will"] or x[-1] in ["?"]))
#     data_df["same_auther"] = (data_df["question_user_name"] == data_df["answer_user_name"]).astype(float)
#     data_df["question_title_selection"] = (data_df["question_title"].apply(lambda x: "or" in x)).astype(float)

#     for i in tqdm(range(len(data_df))):
#         indirect = 0
#         question_count = 0
#         reason_explanation_words = 0
#         choice_words = 0
#         doc = nlp(data_df.loc[i, 'question_body'])
#         for sent in doc.sents:
#             if '?' in sent.text and '?' == sent.text[-1]:
#                 question_count += 1  # -> question_multi_intent
#             for token in sent:
#                 if token.text.lower() == 'why':  # question_type_reason_explanation e.g index->102
#                     reason_explanation_words += 1
#                 elif token.text.lower() == 'or':
#                     choice_words += 1  # question_type_choice
#         if question_count == 0:
#             indirect = 1
#         data_df.loc[i, 'indirect'] = indirect
#         data_df.loc[i, 'question_count'] = question_count
#         data_df.loc[i, 'reason_explanation_words'] = reason_explanation_words
#         data_df.loc[i, 'choice_words'] = choice_words
  
#     data_df['host'] = data_df.host.apply(lambda x: x.split('.')[0])
#     data_df['question_title'] = data_df['category']+' '+data_df['host']+' '+data_df['question_title']

#     return data_df

# train, test = aux_features(train), aux_features(test)
